In [1]:
from utils import load_data, check_gpu, check_dir

import pandas as pd
import numpy as np
from tqdm import tqdm
import gc
from utils import meta_encoding
from functools import partial
import matplotlib.pyplot as plt
from scipy.stats import rankdata
import datetime
import lightgbm as lgb
pd.options.display.max_colwidth = 1000
pd.options.display.max_columns = 1000

%matplotlib inline
%load_ext autoreload
%autoreload 2
def fprint(df, name):
    print(f'{name} shape: ({df.shape[0]:,}, {df.shape[1]})')
import pprint
pp = pprint.PrettyPrinter(indent=1)

In [2]:
train_raw = load_data('train', nrows=1000000)
train_raw['timestamp'] = train_raw['timestamp'].apply(lambda ts: datetime.datetime.utcfromtimestamp(ts))


[06-15 11:48:02 - utils-113 - load_data - INFO] Loading train using 1,000,000 rows (999,995 trimmed) which is 6.28% out of total train data


In [3]:
train_ids = train_raw.session_id.unique()

In [4]:
# look at raw
rid = np.random.choice(train_ids, 1)[0]
rid

'1f6409d88e355'

### Check clean data

In [8]:
train_cleaned = pd.read_parquet('gbm_cache/preprocess_train_100000_no_test_added.snappy')

In [12]:
train_ids = train_cleaned.session_id.unique()

problems: 'b160c4dab0582'

In [13]:
rid = np.random.choice(train_ids, 1)[0]
rid

'517878880bc06'

In [9]:
rid = 'b160c4dab0582'

In [14]:
# raw
raw_demo = train_raw[train_raw.session_id==rid].copy()
raw_demo

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
38351,X4327A0BJV6P,517878880bc06,2018-11-03 12:26:16,1,clickout item,1497671,IN,"Puttaparthi, India",desktop,NaN,1891715|4108080|6059178|4644110|4861306|1497671|1540991|5057838|6544066|4485900|9388226|2701888|4591102|7760844|4649962|8970056|7044990,40|24|16|14|11|9|6|12|18|58|5|27|13|5|7|9|69


In [15]:
# clean
train_cleaned[train_cleaned.session_id==rid]

,session_id,timestamp,step,action_type,current_filters,reference,impressions,prices,device
7635,517878880bc06,2018-11-03 12:26:16,1.0,0,None,1497671,1891715|4108080|6059178|4644110|4861306|1497671|1540991|5057838|6544066|4485900|9388226|2701888|4591102|7760844|4649962|8970056|7044990,40|24|16|14|11|9|6|12|18|58|5|27|13|5|7|9|69,0


### Check feature generations

In [17]:
train_inputs = pd.read_parquet('./gbm_cache/train_inputs_100000_no_test_added.snappy')
cf_cols = [c for c in train_inputs.columns if 'current_filters' in c]
drop_cols = cf_cols  # + ['country', 'platform']
# drop cf col for now
train_inputs.drop(drop_cols, axis=1, inplace=True)

In [ ]:
# # load model 
# clf = lgb.Booster(model_file='./models/lgb_cv0.model')

In [18]:
train_ids = train_inputs.session_id.unique()

In [19]:
rid = np.random.choice(train_ids, 1)[0]
rid

'bc8abdd249ae4'

In [20]:
raw_display_cols = ['timestamp', 'step', 'action_type', 'reference', 'platform', 'city', 'device', 
                    'current_filters', 'impressions', 'prices']

In [21]:
raw_demo = train_raw[train_raw.session_id==rid].reset_index(drop=True)
click_mask = raw_demo.action_type=='clickout item'
answers = raw_demo.loc[click_mask, 'reference']
raw_demo.loc[click_mask, 'reference'] = '?'
raw_demo[raw_display_cols]

,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,2018-11-04 03:11:38,1,interaction item image,3835270,US,"Nassau, Bahamas",mobile,NaN,NaN,NaN
1,2018-11-04 03:11:38,2,interaction item image,3835270,US,"Nassau, Bahamas",mobile,NaN,NaN,NaN
2,2018-11-04 03:12:21,3,interaction item image,3835270,US,"Nassau, Bahamas",mobile,NaN,NaN,NaN
3,2018-11-04 03:12:21,4,interaction item image,3835270,US,"Nassau, Bahamas",mobile,NaN,NaN,NaN
4,2018-11-04 03:12:29,5,interaction item image,3835270,US,"Nassau, Bahamas",mobile,NaN,NaN,NaN
5,2018-11-04 03:12:29,6,interaction item image,3835270,US,"Nassau, Bahamas",mobile,NaN,NaN,NaN
6,2018-11-04 03:12:29,7,interaction item image,3835270,US,"Nassau, Bahamas",mobile,NaN,NaN,NaN
7,2018-11-04 03:12:29,8,interaction item image,3835270,US,"Nassau, Bahamas",mobile,NaN,NaN,NaN
8,2018-11-04 03:12:29,9,interaction item image,3835270,US,"Nassau, Bahamas",mobile,NaN,NaN,NaN
9,2018-11-04 03:12:29,10,interaction item image,3835270,US,"Nassau, Bahamas",mobile,NaN,NaN,NaN


In [26]:
answers

55    3835270
Name: reference, dtype: object

check input data with

In [22]:
input_demo = train_inputs[train_inputs.session_id==rid].reset_index(drop=True)
input_demo

,session_id,step,device,fs,sort_order,imp_changed,session_size,session_duration,last_duration,last_action_type,last_reference_relative_loc,mean_rating,median_rating,mean_star,median_star,n_imps,target,mean_price,median_price,prices_0,prices_1,prices_2,prices_3,prices_4,prices_5,prices_6,prices_7,prices_8,prices_9,prices_10,prices_11,prices_12,prices_13,prices_14,prices_15,prices_16,prices_17,prices_18,prices_19,prices_20,prices_21,prices_22,prices_23,prices_24,prices_rank_0,prices_rank_1,prices_rank_2,prices_rank_3,prices_rank_4,prices_rank_5,prices_rank_6,prices_rank_7,prices_rank_8,prices_rank_9,prices_rank_10,prices_rank_11,prices_rank_12,prices_rank_13,prices_rank_14,prices_rank_15,prices_rank_16,prices_rank_17,prices_rank_18,prices_rank_19,prices_rank_20,prices_rank_21,prices_rank_22,prices_rank_23,prices_rank_24,prev_click_0,prev_click_1,prev_click_2,prev_click_3,prev_click_4,prev_click_5,prev_click_6,prev_click_7,prev_click_8,prev_click_9,prev_click_10,prev_click_11,prev_click_12,prev_click_13,prev_click_14,prev_click_15,prev_click_16,prev_click_17,prev_click_18,prev_click_19,prev_click_20,prev_click_21,prev_click_22,prev_click_23,prev_click_24,prev_interact_0,prev_interact_1,prev_interact_2,prev_interact_3,prev_interact_4,prev_interact_5,prev_interact_6,prev_interact_7,prev_interact_8,prev_interact_9,prev_interact_10,prev_interact_11,prev_interact_12,prev_interact_13,prev_interact_14,prev_interact_15,prev_interact_16,prev_interact_17,prev_interact_18,prev_interact_19,prev_interact_20,prev_interact_21,prev_interact_22,prev_interact_23,prev_interact_24,half_prices_rank_0,half_prices_rank_1,half_prices_rank_2,half_prices_rank_3,half_prices_rank_4,half_prices_rank_5,half_prices_rank_6,half_prices_rank_7,half_prices_rank_8,half_prices_rank_9,half_prices_rank_10,half_prices_rank_11,ratings_0,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,ratings_6,ratings_7,ratings_8,ratings_9,ratings_10,ratings_11,ratings_12,ratings_13,ratings_14,ratings_15,ratings_16,ratings_17,ratings_18,ratings_19,ratings_20,ratings_21,ratings_22,ratings_23,ratings_24,stars_0,stars_1,stars_2,stars_3,stars_4,stars_5,stars_6,stars_7,stars_8,stars_9,stars_10,stars_11,stars_12,stars_13,stars_14,stars_15,stars_16,stars_17,stars_18,stars_19,stars_20,stars_21,stars_22,stars_23,stars_24
0,bc8abdd249ae4,56.0,1,33,8,NaN,9,277.0,27.0,4,0.56,2.08,2.0,2.96,3.0,25,3,255.24,165.0,391.0,159.0,220.0,220.0,645.0,366.0,133.0,91.0,106.0,66.0,157.0,254.0,121.0,294.0,768.0,288.0,165.0,105.0,110.0,395.0,165.0,31.0,645.0,335.0,151.0,0.863636,0.5,0.590909,0.590909,0.954545,0.818182,0.363636,0.136364,0.227273,0.090909,0.454545,0.636364,0.318182,0.727273,1.0,0.681818,0.545455,0.181818,0.272727,0.909091,0.545455,0.045455,0.954545,0.772727,0.409091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.909091,0.545455,0.636364,0.636364,1.0,0.818182,0.363636,0.181818,0.272727,0.090909,0.454545,0.727273,1.0,0.6,0.6,1.0,0.8,0.4,0.2,0.4,0.6,0.4,0.6,0.6,0.8,1.0,1.0,0.4,0.8,0.2,0.2,0.6,0.6,0.2,1.0,0.6,0.8,1.0,0.8,0.6,1.0,0.2,0.8,0.4,0.6,0.6,0.2,0.6,0.8,0.2,0.8,1.0,0.6,0.6,0.4,0.4,0.8,0.8,0.2,0.8,0.8,0.6


In [28]:
14/25

0.56

In [25]:
pp.pprint(input_demo.to_dict())

{'device': {0: 1},
 'fs': {0: 33},
 'half_prices_rank_0': {0: 0.9090909090909091},
 'half_prices_rank_1': {0: 0.5454545454545454},
 'half_prices_rank_10': {0: 0.45454545454545453},
 'half_prices_rank_11': {0: 0.7272727272727273},
 'half_prices_rank_2': {0: 0.6363636363636364},
 'half_prices_rank_3': {0: 0.6363636363636364},
 'half_prices_rank_4': {0: 1.0},
 'half_prices_rank_5': {0: 0.8181818181818182},
 'half_prices_rank_6': {0: 0.36363636363636365},
 'half_prices_rank_7': {0: 0.18181818181818182},
 'half_prices_rank_8': {0: 0.2727272727272727},
 'half_prices_rank_9': {0: 0.09090909090909091},
 'imp_changed': {0: nan},
 'last_action_type': {0: 4},
 'last_duration': {0: 27.0},
 'last_reference_relative_loc': {0: 0.56},
 'mean_price': {0: 255.24},
 'mean_rating': {0: 2.08},
 'mean_star': {0: 2.96},
 'median_price': {0: 165.0},
 'median_rating': {0: 2.0},
 'median_star': {0: 3.0},
 'n_imps': {0: 25},
 'prev_click_0': {0: 0.0},
 'prev_click_1': {0: 0.0},
 'prev_click_10': {0: 0.0},
 'prev

In [ ]:
shap_values[0].shape

In [ ]:
ps = '220|193|310|265|162|119|99|43|51|83|96|65'.split('|')
ps = [float(i) for i in ps]
rankdata(ps, method='dense')

In [ ]:
np.nanmean(ps), np.nanmedian(ps), np.mean(ps), np.median(ps)

In [ ]:
1/12

In [ ]:
2/25

In [ ]:
meta_mapping = meta_encoding()

In [ ]:
# refs = raw_eg.impressions.dropna().str.split('|')
# refs = list([int(j) for i in refs for j in i])
refs = '20720|20814|83606|20772|20752|45909|9844958|20861|20681|45379|2085654|20785|152418|20848|20736|20750|20949|20743|917187|20745|20677|905283|83339|20843|20819'
refs = refs.split('|')
refs = [int(i) for i in refs]

In [ ]:
refs_meta = meta_mapping[meta_mapping.item_id.isin(refs)]
s = refs_meta.sum(axis=0)
refs_meta_useful = refs_meta[s[s!=0].index].reset_index(drop=True)
ss = refs_meta_useful.sum(axis=0)
refs_meta_useful = refs_meta_useful[ss[ss!=24].index]
refs_meta_useful = refs_meta_useful.set_index('item_id')
refs_meta_useful = refs_meta_useful.reindex(refs)
# cols = ['1 star', '2 star', '3 star', '4 star', '5 star', 
#                   'excellent rating', 'very good rating', 'good rating', 'satisfactory rating']
# refs_meta_useful[[c for c in cols if c in refs_meta_useful.columns]].loc[refs]
refs_meta_useful

In [ ]:
(6*4+3*2+2+1)/12, (1+4+4+1+3+2+0+0+2+3+3)/12

### check the meta properties

In [ ]:
meta_mapping = meta_encoding()

In [ ]:
# refs = raw_eg.impressions.dropna().str.split('|')
# refs = list([int(j) for i in refs for j in i])
refs = '14910|52419|53438|15918|15402|128443|772473|626051|158537|1193842|14824|5179324'
refs = refs.split('|')
refs = [int(i) for i in refs]

In [ ]:
refs_meta = meta_mapping[meta_mapping.item_id.isin(refs)]
s = refs_meta.sum(axis=0)
refs_meta_useful = refs_meta[s[s!=0].index].reset_index(drop=True)
ss = refs_meta_useful.sum(axis=0)
refs_meta_useful = refs_meta_useful[ss[ss!=24].index]
refs_meta_useful = refs_meta_useful.set_index('item_id')
refs_meta_useful = refs_meta_useful.reindex(refs)
refs_meta_useful

In [ ]:
cols_sum = refs_meta_useful.sum(axis=0)
one_cols = cols_sum[cols_sum==1].index
refs_meta_useful[one_cols]

In [ ]:
meta_mapping[meta_mapping.item_id==110702]

In [ ]:
m = pd.read_csv('./data/item_metadata.csv')

In [ ]:
m[m.item_id==110702]

In [ ]:
meta_mapping.columns.values

In [ ]:
meta_mapping[['1 star', '2 star', '3 star', 'from 3 stars', '4 star', 'from 4 stars', '5 star']]

In [ ]:
meta_mapping[['satisfactory rating', 'good rating', 'very good rating', 'excellent rating']]

In [ ]:
meta_mapping.head()

In [ ]:
(meta_mapping.loc[meta_mapping['good rating']==1, 'satisfactory rating']==0).sum()

In [ ]:
81/187

In [ ]:
1/25

In [ ]:
s = '81|120|81|187|70|45|56|130|124|56|58|77|52|40|120|53|65|65|54|30|59|33|45|108|37'
s = s.split('|')
s = [int(i) for i in s]
from scipy.stats import rankdata
rankdata(s, method='dense')

In [ ]:
17/20

In [ ]:
# input_demo = train_inputs[train_inputs.session_id==rid].reset_index(drop=True)
# input_demo

In [ ]:
# pp.pprint(input_demo.to_dict(orient='list'))

In [ ]:
from create_model_inputs import change_sort_order_mapping
change_sort_order_mapping()

### country related

In [ ]:
cp = load_data('train', usecols=['city', 'platform'])

In [ ]:
cp['country'] = cp['city'].str.split(', ').str[-1]
cp['country'] = cp['country'].str.lower()

In [ ]:
countries = cp['country'].unique()

In [ ]:
platforms = cp['platform'].unique()

In [ ]:
country2code = np.load('data/country2code.npy').item()

In [ ]:
code2country = np.load('data/code2country.npy').item()

In [ ]:
set(countries) - set(list(country2code.keys()))

In [ ]:
set(list(country2code.keys())) - set(countries)

In [ ]:
fix_dict = {'bermudas': 'bermuda', 
            'bes islands': None,
            'brunei':'brunei darussalam',
            'china': "people's republic of china",
            'crimea': None,
            'curacao': 'netherlands antilles',
            'democratic republic of congo': 'congo, the democratic republic of',
            'east timor': 'timor',
            'french antilles': None,
            'guinea-bissau': None,
            'iran': 'iran, islamic republic of',
            'ivory coast': "côte d'ivoire",
            'kosovo': None,
            'laos': "lao people's democratic republic"
           }

In [ ]:
train_raw.impressions.notna()[train_raw.impressions.notna()].index[0]

In [ ]:
a = pd.read_parquet('./gbm_cache/preprocess_train_5000000_test_added.snappy')

In [ ]:
a.reference.head()

In [ ]:
a.reference.isna().sum()

In [ ]:
a[a.reference.isna()]

In [ ]:
b = pd.read_csv('./data/test.csv')

In [ ]:
b[b.session_id=='2a181b2125efe']

In [ ]:
import datetime
b['timestamp'] = b['timestamp'].apply(lambda ts: datetime.datetime.utcfromtimestamp(ts))


In [ ]:
b.timestamp

In [ ]:
a = pd.DataFrame({'A': np.random.randint(0, 3 ,10), 'B': np.random.randint(0, 5 ,10)})
a.loc[np.random.choice(a.index, 3), 'B'] = np.nan
a

In [ ]:
c = a.groupby('A')['B'].apply(list).reset_index()
c

In [ ]:
c.reset_index().B.values.tolist()

In [ ]:
m = {0: 'E', 1: 'Q'}
a.A.map(m)